In [1]:
from preprocessing import extract_text_and_labels, tokenizer, get_sequences
from model import model_0
from utils import import_data, show_confusion_matrix, show_history, get_index

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp 
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import ProgbarLogger

2023-09-15 19:18:22.935014: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 19:18:22.936500: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 19:18:22.970630: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 19:18:22.971051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 19:18:23.600711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:

train_path = './dataset/training.csv'
valid_path = './dataset/validation.csv'

train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)

train_text, train_labels = extract_text_and_labels(train)
valid_text, valid_labels = extract_text_and_labels(valid)

tokenizer_train, train_tok = tokenizer(train_text)
tokenizer_valid, valid_tok = tokenizer(valid_text)

max_len = 50 # max sequence length

padded_train_seq = get_sequences(tokenizer_train, train_tok, max_len)
padded_valid_seq = get_sequences(tokenizer_valid, valid_tok, max_len)

class_to_index, index_to_class = get_index()

train_labels = np.array(train_labels)
valid_labels = np.array(valid_labels)

In [ ]:

model = model_0()

checkpoint_callback = ModelCheckpoint(
    filepath='weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1 # Display messages when saving weights
)

hypothesis = model.fit(
    x=padded_train_seq,
    y=train_labels,
    validation_data=(padded_valid_seq, valid_labels),
    epochs=100,
    callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=2),
        checkpoint_callback
    ],
    verbose=1 
)

model.save('Model.h5')

In [9]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

# Load the trained model
model = load_model('./model/Model.h5')

# Create a function to preprocess text input and make predictions
def predict_emotion():
    user_input = text_entry.get()  # Get text input from the user
    if user_input:
        # Tokenize and preprocess the input text
        user_seq = tokenizer_train.texts_to_sequences([user_input])
        padded_user_seq = pad_sequences(user_seq, truncating='post', padding='post', maxlen=max_len)
        
        # Make a prediction using the loaded model
        prediction = model.predict(padded_user_seq)
        
        # Get the emotion label with the highest probability
        predicted_label = index_to_class[np.argmax(prediction)]
        
        # Display the result in a message box
        result_message = f"Predicted Emotion: {predicted_label}"
        messagebox.showinfo("Emotion Prediction Result", result_message)
    else:
        messagebox.showwarning("Empty Input", "Please enter text for prediction.")

# Create the main GUI window
root = tk.Tk()
root.title("Emotion Prediction")

# Create a label and text entry field
label = ttk.Label(root, text="Enter Text:")
label.pack(pady=10)
text_entry = ttk.Entry(root, width=40)
text_entry.pack(pady=5)

# Create a button to make predictions
predict_button = ttk.Button(root, text="Predict Emotion", command=predict_emotion)
predict_button.pack(pady=10)

# Start the GUI application
root.mainloop()


1/1 [==============================] - 1s 764ms/step
